# MLflow's Model Registry

In [1]:
# from mlflow.tracking import MlflowClient
from mlflow import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

## Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with:
* an MLflow Tracking Server that creates and manages experiments and runs.
* an MLflow Registry Server that creates and manages registered models and model versions.

To instantiate it we need to pass a tracking URI and/or a registry URI.

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

Now that we've instantiated the client object, we can access the tracking server data, such as our experiments.

In [4]:
client.search_experiments()

[<Experiment: artifact_location='/home/sgrodriguez/Formación/mlops_dtc/02_experiment_tracking/mlruns/1', creation_time=1684794410751, experiment_id='1', last_update_time=1684794410751, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1684792727462, experiment_id='0', last_update_time=1684792727462, lifecycle_stage='active', name='Default', tags={}>]

We can also create new experiments (once it's created, it can be found in the MLflow UI).

In [7]:
client.create_experiment(name="some_new_experiment")

'3'

Let's check the latest versions for the experiment with id `1`.

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

runs

[<Run: data=<RunData: metrics={'rmse': 6.137911990367575}, params={'learning_rate': '0.1126860623846719',
  'max_depth': '11',
  'min_child_weight': '7.128461099684721',
  'objective': 'reg:linear',
  'reg_alpha': '0.04429046957254972',
  'reg_lambda': '0.09902356874800584',
  'seed': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "2dba47c1349d4939af149471c6129149", '
                              '"artifact_path": "models_mlflow", '
                              '"utc_time_created": "2023-05-27 '
                              '10:03:01.510865", "flavors": {"python_function": '
                              '{"loader_module": "mlflow.xgboost", '
                              '"python_version": "3.9.16", "data": "model.xgb", '
                              '"env": {"conda": "conda.yaml", "virtualenv": '
                              '"python_env.yaml"}}, "xgboost": {"xgb_version": '
                              '"1.7.5", "data": "model.xgb", "model_class": '
                    

In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 2dba47c1349d4939af149471c6129149, rmse: 6.1379
run id: 47d279fdf909439b8e1199f394ecab96, rmse: 6.1379
run id: 16026d914a4541cb8a0ec4a314ffaee9, rmse: 6.1386
run id: ed79f054309642198793072a8cf4184f, rmse: 6.1419
run id: 3d12ed80ba3c42db9fb623c56dd4abfe, rmse: 6.1431


## Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:
1. Register a new version for the experiment `nyc_taxi_regressor`.
2. Retrieve the latests versions of the model `nyc_taxi_regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [10]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

We can programatically promote a model to the registry. We don't actually need the `client` object to do this.

In [11]:
run_id = "ed79f054309642198793072a8cf4184f"
model_uri = f"runs:/{run_id}/model"

mlflow.register_model(model_uri=model_uri, name="nyc_taxi_regressor")

Registered model 'nyc_taxi_regressor' already exists. Creating a new version of this model...
2023/05/27 19:45:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc_taxi_regressor, version 4
Created version '4' of model 'nyc_taxi_regressor'.


<ModelVersion: aliases=[], creation_timestamp=1685209527883, current_stage='None', description=None, last_updated_timestamp=1685209527883, name='nyc_taxi_regressor', run_id='ed79f054309642198793072a8cf4184f', run_link=None, source='/home/sgrodriguez/Formación/mlops_dtc/02_experiment_tracking/mlruns/1/ed79f054309642198793072a8cf4184f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [12]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1685183025573, description='NYC Taxi predictor for trip duration.', last_updated_timestamp=1685209527883, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1685183089749, current_stage='Staging', description='', last_updated_timestamp=1685184477587, name='nyc_taxi_regressor', run_id='2dba47c1349d4939af149471c6129149', run_link='', source='/home/sgrodriguez/Formación/mlops_dtc/02_experiment_tracking/mlruns/1/2dba47c1349d4939af149471c6129149/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=2>,
  <ModelVersion: aliases=[], creation_timestamp=1685209527883, current_stage='None', description=None, last_updated_timestamp=1685209527883, name='nyc_taxi_regressor', run_id='ed79f054309642198793072a8cf4184f', run_link=None, source='/home/sgrodriguez/Formación/mlops_dtc/02_experiment_tracking/mlruns/1/ed79f054309642198793072a8cf4184f/artifacts/model', status='READY', status

In [15]:
model_name = "nyc_taxi_regressor"

latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: Production
version: 4, stage: None


Let's transition version `4` to `Staging`.

In [16]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685209527883, current_stage='Staging', description=None, last_updated_timestamp=1685209957756, name='nyc_taxi_regressor', run_id='ed79f054309642198793072a8cf4184f', run_link=None, source='/home/sgrodriguez/Formación/mlops_dtc/02_experiment_tracking/mlruns/1/ed79f054309642198793072a8cf4184f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

We can also make annotations to our model version, such as a description.

In [17]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685209527883, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2023-05-28', last_updated_timestamp=1685265511402, name='nyc_taxi_regressor', run_id='ed79f054309642198793072a8cf4184f', run_link=None, source='/home/sgrodriguez/Formación/mlops_dtc/02_experiment_tracking/mlruns/1/ed79f054309642198793072a8cf4184f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

## Comparing versions and selecting the new Production model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:
1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.

**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [21]:
from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ["PULocationID", "DOLocationID"]
    df.loc[:, categorical] = df.loc[:, categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [22]:
!wget -NP ./data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet

--2023-05-28 11:30:00--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet
Resolviendo d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.67.246.167, 18.67.246.47, 18.67.246.186, ...
Conectando con d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)[18.67.246.167]:443... conectado.
Petición HTTP enviada, esperando respuesta... 304 Not Modified
Fichero “./data/green_tripdata_2021-03.parquet” no modificado en el servidor. Omitiendo la descarga.



In [24]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [26]:
run_id = "2dba47c1349d4939af149471c6129149"

client.download_artifacts(run_id=run_id, path="preprocessor", dst_path=".")

/tmp/ipykernel_46562/908814849.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path="preprocessor", dst_path=".")


'/home/sgrodriguez/Formación/mlops_dtc/02_experiment_tracking/preprocessor'

In [27]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [28]:
X_test = preprocess(df, dv)

In [29]:
target = "duration"
y_test = df[target].values

In [30]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

2023/05/28 11:39:03 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[11:39:03] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 2.23 s, sys: 5.17 ms, total: 2.23 s
Wall time: 403 ms


{'rmse': 6.770245690879554}

In [32]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

2023/05/28 11:45:25 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 107 ms, sys: 7.83 ms, total: 115 ms
Wall time: 138 ms


{'rmse': 6.773844188699355}

Once we've got our results, we can decide which model to keep and use `transition_model_version_stage()` to transition the chosen model to production while archiving any previous production models.

In [33]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1685183025601, current_stage='Production', description='', last_updated_timestamp=1685267197488, name='nyc_taxi_regressor', run_id='87d36b30e49d4f0fbc54d4a5e4660e27', run_link='', source='/home/sgrodriguez/Formación/mlops_dtc/02_experiment_tracking/mlruns/1/87d36b30e49d4f0fbc54d4a5e4660e27/artifacts/model', status='READY', status_message=None, tags={'model': 'gradientboostingregressor'}, user_id=None, version=1>